In [3]:
import os
import sys

spark_path = os.environ['SPARK_HOME']
sys.path.append(spark_path + "/bin")
sys.path.append(spark_path + "/python")
sys.path.append(spark_path + "/python/pyspark/")
sys.path.append(spark_path + "/python/lib")
sys.path.append(spark_path + "/python/lib/pyspark.zip")
sys.path.append(spark_path + "/python/lib/py4j-0.10.9-src.zip")

import findspark
findspark.init()

import pyspark

number_cores = 4
memory_gb = 8

conf = (pyspark.SparkConf().setMaster('local[{}]'.format(number_cores)).set('spark.driver.memory', '{}g'.format(memory_gb)))

conf

sc = pyspark.SparkContext(conf=conf)

In [58]:
graph = sc.textFile("./sm_graph.dat")

In [83]:
!ls -l ./sm_graph.dat

-rw-r--r--  1 adrian  staff  19 Sep 26 10:45 ./sm_graph.dat


In [84]:
graph.take(10)

['y y', 'y a', 'a y', 'a m', 'm a']

Matrix formulation

In [85]:
links = graph.map( lambda line: (line.split(" ")[0], line.split(' ')[1])).groupByKey().mapValues(list)
links.take(10)

[('y', ['y', 'a']), ('a', ['y', 'm']), ('m', ['a'])]

In [86]:
N = links.count()
ranks = links.map(lambda line: (line[0], 1/N) )
ranks.take(N)

[('y', 0.3333333333333333),
 ('a', 0.3333333333333333),
 ('m', 0.3333333333333333)]

In [87]:
votes = ranks.join(links)
votes.take(N)

[('y', (0.3333333333333333, ['y', 'a'])),
 ('a', (0.3333333333333333, ['y', 'm'])),
 ('m', (0.3333333333333333, ['a']))]

In [88]:
def calculateVotes(t):
    res = []
    for item in t[1][1]:
        count = len(t[1][1])
        res.append((item, t[1][0] / count))
    return res

calculate_votes( ('y',(0.3333333333333333, ['y','a'])) )

[('y', 0.16666666666666666), ('a', 0.16666666666666666)]

In [89]:
votes.take(5)

[('y', (0.3333333333333333, ['y', 'a'])),
 ('a', (0.3333333333333333, ['y', 'm'])),
 ('m', (0.3333333333333333, ['a']))]

In [72]:
votes = ranks.flatMap(calculate_votes)
votes.collect()

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.collectAndServe.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 1 in stage 242.0 failed 1 times, most recent failure: Lost task 1.0 in stage 242.0 (TID 321, 104.201.133.117, executor driver): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/users/adrian/spark-3.0.1-bin-hadoop3.2/python/lib/pyspark.zip/pyspark/worker.py", line 605, in main
    process()
  File "/users/adrian/spark-3.0.1-bin-hadoop3.2/python/lib/pyspark.zip/pyspark/worker.py", line 597, in process
    serializer.dump_stream(out_iter, outfile)
  File "/users/adrian/spark-3.0.1-bin-hadoop3.2/python/lib/pyspark.zip/pyspark/serializers.py", line 271, in dump_stream
    vs = list(itertools.islice(iterator, batch))
  File "/users/adrian/spark-3.0.1-bin-hadoop3.2/python/lib/pyspark.zip/pyspark/util.py", line 107, in wrapper
    return f(*args, **kwargs)
  File "<ipython-input-

In [50]:
ranks = votes.reduceByKey(lambda x, y: x + y)
ranks.take(N)

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.runJob.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 1 in stage 161.0 failed 1 times, most recent failure: Lost task 1.0 in stage 161.0 (TID 230, 104.201.133.117, executor driver): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/users/adrian/spark-3.0.1-bin-hadoop3.2/python/lib/pyspark.zip/pyspark/worker.py", line 605, in main
    process()
  File "/users/adrian/spark-3.0.1-bin-hadoop3.2/python/lib/pyspark.zip/pyspark/worker.py", line 595, in process
    out_iter = func(split_index, iterator)
  File "/users/adrian/spark-3.0.1-bin-hadoop3.2/python/pyspark/rdd.py", line 2596, in pipeline_func
    return func(split, prev_func(split, iterator))
  File "/users/adrian/spark-3.0.1-bin-hadoop3.2/python/pyspark/rdd.py", line 2596, in pipeline_func
    return func(split, prev_func(split, iterator))
  File "/users/adrian/spark-3.0.1-bin-hadoo

In [80]:
ranks.values().sum()

1.0

In [81]:
%%time
for i in range(10):
    votes = ranks.flatmap(calculate_votes)
    ranks = votes.reduceByKey(lambda x, y: x + y)
    print(ranks.take(N))

AttributeError: 'PipelinedRDD' object has no attribute 'flatmap'

In [82]:
sum = 1 
while sum > 0.1:
    old_ranks = ranks
    votes = ranks.flatmap(calculate_votes)
    ranks = votes.reduceByKey(lambda x, y: x + y)
    errors = old_ranks.join(ranks).mapValues(lambda v: abs(v[0] - v[1]))
    sum = errors.values().sum()
    print(sum)

AttributeError: 'PipelinedRDD' object has no attribute 'flatmap'

Hollins Dataset

In [120]:
!wc -l ./hollins.dat

   29888 ./hollins.dat


In [121]:
raw_data = sc.textFile('./hollins.dat')

In [122]:
raw_data.take(50)

['6012 23875',
 '1 http://www1.hollins.edu/ ',
 '2 http://www.hollins.edu/ ',
 '3 http://www1.hollins.edu/Docs/CompTech/Network/webmail_faq.htm ',
 '4 http://www1.hollins.edu/Docs/Forms/GetForms.htm ',
 '5 http://www1.hollins.edu/Docs/misc/travel.htm ',
 '6 http://www1.hollins.edu/Docs/GVCalendar/gvmain.htm ',
 '7 http://www1.hollins.edu/docs/events/events.htm ',
 '8 http://www1.hollins.edu/docs/comptech/mainviruses.htm ',
 '9 http://www1.hollins.edu/Docs/Academics/acad.htm ',
 '10 http://www1.hollins.edu/Docs/CompTech/Blackboard/bb_faq.htm ',
 '11 http://www1.hollins.edu/Docs/comptech/comptech.htm ',
 '12 http://www1.hollins.edu/Docs/Academics/international_programs/index.htm ',
 '13 http://www1.hollins.edu/Docs/academics/online/cyber.htm ',
 '14 http://www1.hollins.edu/Registrar/registrar.htm ',
 '15 http://www1.hollins.edu/Docs/Academics/writingcenter/wcenter.htm ',
 '16 http://www.hollins.edu/about/map/map.htm ',
 '17 http://www1.hollins.edu/Docs/SchoolServices/FoodServ/default.htm

In [123]:
lc = raw_data.count()
print(lc)

29888


In [124]:
header_vals = [ int(x) for x in raw_data.first().split() ]
header_vals[0] + header_vals[1]

29887

In [133]:
raw_data.takeSample(False, 50)

['5378 5653',
 '624 27',
 '2081 880',
 '649 655',
 '3587 http://www1.hollins.edu/classes/dance/website/vanessaweb/resume.html ',
 '1208 1210',
 '297 252',
 '5184 3950',
 '45 294',
 '775 176',
 '1877 822',
 '1544 66',
 '3236 3252',
 '161 157',
 '5924 4806',
 '5380 5711',
 '3941 3974',
 '798 http://www.hollins.edu/athletics/volleyball/schedule.htm ',
 '5504 5884',
 '1188 446',
 '2127 3157',
 '4736 http://www1.hollins.edu/registrar/MAJORS-MINORS/MAJ-MIN%2002-03/HISTORY%20MINOR.pdf ',
 '2536 3326',
 '2829 http://www1.hollins.edu/docs/gvcalendar/gvarchive/gvjuly23.htm ',
 '1027 http://www1.hollins.edu/ssforms/Advising.asp ',
 '1019 http://www.hollins.edu/undergrad/womenstudies/womfac.htm ',
 '5401 5397',
 '74 38',
 '3601 http://www1.hollins.edu/classes/dance/website/nicole/1.html ',
 '222 890',
 '880 886',
 '4024 4023',
 '818 37',
 '1410 2460',
 '522 http://www.hollins.edu/admissions/ugradadm/international/map2.htm ',
 '1208 69',
 '21 92',
 '3884 http://www1.hollins.edu/faculty/saloweyca/cl

In [134]:
def is_numeric(n):
    try:
        tmp = int(n)
        return True
    except:
        return False

In [135]:
num_num = raw_data.filter(lambda line: is_numeric(line.split(" ")[1]))
num_num.count()

23876

In [137]:
num_text = raw_data.filter(lambda line: not is_numeric(line.split(" ")[1]))
num_text.count()

6012

In [138]:
links = num_num.map(lambda line: (line.split(" ")[0], line.split(" ")[1])) \
        .groupByKey() \
        .mapValues(list)
links.take(10)

[('1',
  ['2',
   '3',
   '4',
   '5',
   '6',
   '7',
   '8',
   '9',
   '10',
   '11',
   '12',
   '13',
   '14',
   '15',
   '16',
   '17',
   '18',
   '19',
   '20',
   '21',
   '22',
   '23',
   '24',
   '25']),
 ('8',
  ['2',
   '7',
   '19',
   '21',
   '85',
   '87',
   '88',
   '90',
   '91',
   '92',
   '221',
   '222',
   '223',
   '224',
   '225',
   '226',
   '227',
   '228',
   '229',
   '230',
   '340',
   '341',
   '342',
   '343']),
 ('16',
  ['2',
   '27',
   '28',
   '37',
   '38',
   '42',
   '48',
   '52',
   '61',
   '73',
   '74',
   '75',
   '76',
   '77',
   '78',
   '79']),
 ('20',
  ['2',
   '217',
   '248',
   '482',
   '483',
   '484',
   '485',
   '486',
   '487',
   '488',
   '489',
   '490',
   '491',
   '492',
   '493']),
 ('26',
  ['2', '27', '28', '37', '38', '39', '40', '41', '42', '43', '52', '61']),
 ('29',
  ['2',
   '27',
   '28',
   '37',
   '38',
   '43',
   '52',
   '61',
   '73',
   '118',
   '119',
   '120',
   '121',
   '122',
   '123',
   

In [139]:
N = links.count()
ranks = links.map(lambda line: (line[0], 1 / N))
ranks.take(10)

[('1', 0.0003541076487252125),
 ('8', 0.0003541076487252125),
 ('16', 0.0003541076487252125),
 ('20', 0.0003541076487252125),
 ('26', 0.0003541076487252125),
 ('29', 0.0003541076487252125),
 ('33', 0.0003541076487252125),
 ('34', 0.0003541076487252125),
 ('44', 0.0003541076487252125),
 ('45', 0.0003541076487252125)]

In [140]:
def calculateVotes(t):
    res = []
    for item in t[1][1]:
        count = len(t[1][1])
        res.append((item, t[1][0] / count))
    return res
calculateVotes(('y', (0.3333333333333333, ['y', 'a'])))

[('y', 0.16666666666666666), ('a', 0.16666666666666666)]

In [141]:
%%time
sum = 1
N = links.count()
ranks = links.map(lambda line: (line[0], 1 / N))
ranks.take(10)

while sum > 0.1:
    old_ranks = ranks
    votes = ranks.join(links) \
        .flatMap(calculateVotes)
    ranks = votes.reduceByKey(lambda x, y: x + y)
    errors = old_ranks.join(ranks).mapValues(lambda v: abs(v[0] - v[1]))
    sum = errors.values().sum()
    print(sum)

1.0289126892749134
0.6524166912065938
0.354732060058874
0.20752688756812687
0.14669618513501703
0.11380517976401004
0.09642834963258069
CPU times: user 312 ms, sys: 60.7 ms, total: 373 ms
Wall time: 4.4 s


In [144]:
ranks.takeOrdered(10,key = lambda x: -x[1])

[('2', 0.01480246014081635),
 ('37', 0.010738970689076617),
 ('38', 0.009867983441807206),
 ('61', 0.009734731227889629),
 ('52', 0.009357107790050318),
 ('425', 0.00868217558595644),
 ('43', 0.008005883519059315),
 ('27', 0.00761747437182122),
 ('28', 0.006066068156415338),
 ('29', 0.0056368097448235835)]